### importing and reading

In [1]:
# import modules
import pandas as pd
import seaborn as sns
import numpy as np

# Fixing random state for reproducibility
np.random.seed(387)

In [9]:
# read data
characters = pd.read_csv('datasets/Characters.csv', sep=';')
hp1 = pd.read_csv('datasets/Harry Potter 1.csv', sep=';')
hp2 = pd.read_csv('datasets/Harry Potter 2.csv', sep=';')
hp3 = pd.read_csv('datasets/Harry Potter 3.csv', sep=';')

### getting column names, and table shapes

In [10]:
# column names
print(characters.columns,'\n')
print(hp1.columns)

Index(['Id', 'Name', 'Gender', 'Job', 'House', 'Wand', 'Patronus', 'Species',
       'Blood status', 'Hair colour', 'Eye colour', 'Loyalty', 'Skills',
       'Birth', 'Death'],
      dtype='object') 

Index(['Character', 'Sentence'], dtype='object')


In [11]:
# 3 tables shapes
print("hp1 shape:", hp1.shape,'\n',
      "hp2 shape:", hp2.shape,'\n',
      "hp3 shape:", hp3.shape)

hp1 shape: (1587, 2) 
 hp2 shape: (1700, 2) 
 hp3 shape: (1638, 2)


# Plots :
### Characters with the most sentences
#### top 15 of the first 3 movies 

In [25]:
# vertical append
combined = pd.concat([hp1,hp2], ignore_index=True)
combined = pd.concat([combined,hp3], ignore_index=True)
combined

,Character,Sentence,CHARACTER,SENTENCE
0,Dumbledore,"I should've known that you would be here, Prof...",NaN,NaN
1,McGonagall,"Good evening, Professor Dumbledore.",NaN,NaN
2,McGonagall,"Are the rumors true, Albus?",NaN,NaN
3,Dumbledore,"I'm afraid so, professor.",NaN,NaN
4,Dumbledore,The good and the bad.,NaN,NaN
...,...,...,...,...
4920,NaN,NaN,HERMIONE,"How fast is it, Harry?"
4921,NaN,NaN,HARRY,Lumos.
4922,NaN,NaN,HARRY,I solemnly swear that I am up to no good.
4923,NaN,NaN,HARRY,Mischief managed.
